# Progetto FAD 25-26

Progetto di gruppo per il corso di *Fondamenti di Analisi dei Dati* — A.A. 2025/2026

**Nome del gruppo**: *The Bayesians*

**Membri del gruppo**:
- *Filippo Falzone*  
- *Francesco Greco*

Il progetto si basa sullo studio di un dataset contenente informazioni circa varie automobili vendute.
I dati sono stati raccolti in India e le auto vendute sono usate. Quindi lo studio si baserà sul comprendere il mercato delle automobili usate in India.



---

## 1. Analisi dei dati

Questa fase iniziale è dedicata alla **comprensione approfondita del dataset**. Si procede con l'esplorazione, la pulizia dei dati (data cleaning) per gestire valori mancanti e anomalie, e l'analisi descrittiva per inferire le prime caratteristiche e tendenze chiave delle variabili.

### Data Understanding

Ci concentreremo sull'iniziare a conoscere i dataset, comprendendo a fonndo il significato di ogni variabile e formulando alcune domande su cui strutturare le successive fasi dell'analisi dei dati.

In [213]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('../../data_set/train-data.csv', index_col=0)

# 'df_clean' coneterrà il dataset pulito
df_clean = df.copy()

df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


Vediamolo nel dettaglio:

In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6019 entries, 0 to 6018
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               6019 non-null   object 
 1   Location           6019 non-null   object 
 2   Year               6019 non-null   int64  
 3   Kilometers_Driven  6019 non-null   int64  
 4   Fuel_Type          6019 non-null   object 
 5   Transmission       6019 non-null   object 
 6   Owner_Type         6019 non-null   object 
 7   Mileage            6017 non-null   object 
 8   Engine             5983 non-null   object 
 9   Power              5983 non-null   object 
 10  Seats              5977 non-null   float64
 11  New_Price          824 non-null    object 
 12  Price              6019 non-null   float64
dtypes: float64(2), int64(2), object(9)
memory usage: 658.3+ KB


> Il nostro dataset ha **13 caratteristiche** (colonne) e **6019 osservazioni** (righe).

Adesso creiamo un dizionario dei dati, in modo da avere una comprensione maggiore delle varie caratteristiche.

**Dizionario dei Dati**
| Variabile             | Tipo / Scala                     | Descrizione                                                                                         | Unità / Note |
| --------------------- | -------------------------------- | --------------------------------------------------------------------------------------------------- | ------------ |
| **Name**              | Qualitativa / Nominale           | Combinazione di marchio e modello dell’auto. | –            |
| **Location**          | Qualitativa / Nominale           | Città o regione in cui l’auto è in vendita.                                                         | –            |
| **Year**              | Numerica / Discreta / Intervalli | Anno di produzione del modello.                | –            |
| **Kilometers_Driven** | Numerica / Continua / Ratio      | Chilometri totali percorsi dall’auto.                                                               | km           |
| **Fuel_type**         | Qualitativa / Nominale           | Tipo di carburante utilizzato dall’auto.                                                            | –            |
| **Transmission**      | Qualitativa / Nominale            | Tipo di trasmissione: `automatica` o `manuale`.                                                         | –            |
| **Owner_Type**        | Qualitativa / Ordinale           | Indica il numero di proprietari precedenti che il veicolo ha avuto prima del venditore attuale. Le categorie sono ordinate per numero crescente di proprietari: `first_hand` (il venditore è il primo proprietario), `second_hand`, `third_hand`, e `fourth_above` (quattro o più proprietari).                            | –            |
| **Mileage**           | Numerica / Continua / Ratio      | Consumo di carburante dell’auto.                                               | kmpl o km/kg |
| **Engine**            | Numerica / Continua / Ratio      | Cilindrata del motore.                                                                              | cc           |
| **Power**             | Numerica / Continua / Ratio      | Potenza massima del motore.                                                                         | bhp          |
| **Seats**             | Numerica / Discreta / Ratio      | Numero di posti a sedere.            | –            |
| **New_Price**         | Numerica / Continua / Ratio      | Rappresenta il prezzo di un auto nuova dello stesso modello.                                                         | INR Lakhs    |
| **Price**             | Numerica / Continua / Ratio      | Prezzo di vendita dell’auto usata.                 | INR Lakhs    |


Alcune considerazioni sulle variabili:
- da `Name` potremmo estrarre Marca dell'auto e modello
- da `Year` possiamo calcolare l'età dell'auto
- `Transmission` è una variabile binaria (ha solo due possibili categorie)
- modifica di `Owner_Type` in modo da renderla discreta ordinale
- `Seats` può aiutare a classificare il veicolo (SUV, sportiva, etc...)
- individuiamo `Price` come variabile target da predire per nuove osservazioni

**Obiettivi dell'Analisi dei Dati**


Dopo aver descritto e compreso le varie caratteristiche del dataset abbiamo formulato alcuni **obiettivi** su cui basare la nostra analisi dei dati, che ricordiamo basarsi sullo studio del mercato di auto usate in India:
- Quali sono i fattori principali che determinano il prezzo (`Price`) di un'auto usata? 
- In che modo l'età dell'auto (derivata da `Year`) e i chilometri percorsi (`Kilometers_Driven`) interagiscono nel definire il valore?
- Quale impatto hanno le caratteristiche tecniche come il motore o la potenza (`Engine` o `Power`) e il tipo di cambio (`Transmission`) sul prezzo?
- Esistono marche (estratte da `Name`) o località (`Location`) specifiche in cui le auto mantengono meglio il loro valore?  

### Data Cleaning & Preparation

È molto importate preparare il dataset prima di poter iniziare ad analizzare le variabili e come essere sono correlate. Questo perché eventuali valori mancanti o anomalie (outlier) ci porterebbo a conclusioni errate o inconsistenti.

Cominciamo dando un occhiata ai **valori mancanti** (Nan) presenti nelle nostre colonne:

In [215]:
print(df.isna().sum())

print("")

print("---- Nan percentage per variable ----")
for x in df:
    if df[x].isnull().sum() > 0:
        value = (df[x].isna().sum()/len(df[x])) * 100
        print(f"{x}: {value:.2f}%")

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  42
New_Price            5195
Price                   0
dtype: int64

---- Nan percentage per variable ----
Mileage: 0.03%
Engine: 0.60%
Power: 0.60%
Seats: 0.70%
New_Price: 86.31%


Vediamo che la maggior parte delle colonne contengono una **quantità irrilevanti di valori mancati**. Quelli che sono presenti (i missing values) si possono sistemare facilmente: o eliminiamo quelle righe, oppure facendp un'imputazione veloce utilizzando la media o la mediana.

Tuttavia `New_Price` è un **disastro**: 5195 su 6019 righe sono vuote. Parliamo dell'86% dei dati.

- **Imputazione**: impossibile, Il prezzo di listino dello stesso modello è troppo specifico; non possiamo stimarlo tramite le caratteristiche presenti in questo dataset. Non è come stimare il chilometraggio, questo è un valore che dipende da altre variabili.

- **Inutilizzabile**: Avere così tanti buchi rende la colonna inutile sia per un analisi e sia per una modellazione. Non possiamo farci affidamento.

Verifichiamo se nel nostro dataset sono presenti **righe duplicate**, che porterebbero ad un analisi non corretta, e nel caso eliminarle:

In [216]:
duplicate_count = df.duplicated().sum()

df.drop_duplicates(inplace=True)

print(f"Rimosse di n. righe duplicate: {duplicate_count}")


Rimosse di n. righe duplicate: 0


Adesso eseguiremo per ogni caratterisitca un processo di Data Cleaning che includerà:
- gestione dei Valori Mancanti
- Correzzione di Inconsistenze e Formattazione 
- Gestione degli outliers

**1. `Name`**

Ricordiamo che 'Name' contiene il brand e il modello dell'auto in osservazione.

> Contiene una stringa '\<brand> <modello>'

Inoltre, come abbiamo verificato poco fa, non contiene valori mancanti.

*Correzzione di Inconsistenze e Formattazione*

Per la variabile 'Name', che include sia il brand che il modello del veicolo, abbiamo deciso di applicare la Feature Engineering per scomporre questa stringa in due nuove variabili distinte: Brand e Model. Questo perché, anche se la variabile 'Name' è pulita, l'analisi e la modellazione predittiva traggono molti benefici da variabili con un unico scopo ben definito. 

Creeremo quindi le due nuove variabili Brand e Model estraendole dalla stringa originale.

In [217]:
df_clean['Brand'] = df_clean['Name'].str.split(' ').str[0]
df_clean['Model'] = df_clean['Name'].str.split(' ').str[1]
df_clean = df_clean.drop('Name', axis=1)

first_cols = ['Brand', 'Model']

remaining_cols = [col for col in df_clean.columns if col not in first_cols]

df_clean = df_clean[first_cols + remaining_cols]

print("Model e Brand creati e inseriti come prime due colonne.")

df_clean.head()

Model e Brand creati e inseriti come prime due colonne.


,Brand,Model,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti,Wagon,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai,Creta,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda,Jazz,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti,Ertiga,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi,A4,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


**2. `Location`**

Ricordiamo che 'Location' contiene il luogo in cui l'auto è stata venduta.

> Contiene una stringa \<location>

Come abbiamo verificato prima 'Location' non contiene valori mancanti.

Non sono necessari controlli su questa variabile, potremmo controllare se i nomi dei luoghi assunti da essa **esistono veramente**. Lo faremo ma solo per ottenere **informazioni circa i luoghi** in cui queste auto sono state rivendute:

In [218]:
print(df_clean['Location'].unique())
print("")
print(df_clean['Location'].value_counts())

['Mumbai' 'Pune' 'Chennai' 'Coimbatore' 'Hyderabad' 'Jaipur' 'Kochi'
 'Kolkata' 'Delhi' 'Bangalore' 'Ahmedabad']

Location
Mumbai        790
Hyderabad     742
Kochi         651
Coimbatore    636
Pune          622
Delhi         554
Kolkata       535
Chennai       494
Jaipur        413
Bangalore     358
Ahmedabad     224
Name: count, dtype: int64


Abbiamo esaminato i luoghi riportati sopra, non solo essi non sono frutto di errore o fantasia, ma sono tutte **grandi città metropolitane** indiane.

**3. `Year`**

Questa variabile indica l'anno in cui l'auto è stata prodotta.

> Contiene un valore discreto.

Come visto prima non contiene valori mancanti.

*Correzzione di Inconsistenze e Formattazione*

Per questa variabile non servono particolari controlli di questo tipo. Ma possiamo applicare pure qui un po di Feature Engeneering per trarre l'età dell'auto. Questo perché a noi non interessa sapere l'anno di fabbricazione, ma è l'età dell'auto che influisce sul prezzo.

Creeremo quindi la variabile `Age`:

In [219]:
max_year = df_clean['Year'].max()
index_year = df_clean.columns.get_loc('Year')
age_values = max_year - df_clean['Year']
df_clean.insert(index_year, 'Age', age_values)
df_clean.drop('Year', axis=1, inplace=True)

print('Age creata con successo.')

Age creata con successo.


**`Fuel_Type`**

'Fuel_Type' contiene il tipo di carburante utilizzato dall'auto.

> Contiene una stringa '\<tipo_di_carburante>'

Come abbiamo verificato prima 'Location' non contiene valori mancanti.

*Correzzione di Inconsistenze e Formattazione*

Facciamo un controllo sul tipo di valori che può assumere questa variabile:

In [220]:
df_clean['Fuel_Type'].value_counts()

Fuel_Type
Diesel      3205
Petrol      2746
CNG           56
LPG           10
Electric       2
Name: count, dtype: int64

Vediamo che la maggior parte delle auto vendute utilizzano Diesel o Petrol, poi ci sono alcune nicchie. 

La presenza di auto elettriche è un po' problematica. La poca presenza nel dataset potrebbe indicare che c'è **poco mercato** nella regione di studio per auto del genere.

Abbiamo comunque **troppe poche informazioni** per uno studio più approfondito che includa anche le auto **elettriche**, quindi la nostra scelta è quella di **eliminare** questa tipologia di veicoli dal nostro dataset.

In [221]:
# Rimuoviamo le auto elettriche
df_clean = df_clean[df_clean['Fuel_Type'] != 'Electric']
print("Auto elettriche rimosse")

Auto elettriche rimosse


**5. `Mileage`**

> Contiene una **stringa** forma '\<valore\> \<unità di misura\>'

Vogliamo trasformare mileage in una variabile numerica, estraendo il valore dalla stringa.

Verifichiamo il numero di valori mancanti:

In [222]:
print("Valori mancanti:", df_clean['Mileage'].isna().sum())

Valori mancanti: 0


Notiamo come Mileage inizialmente aveva 2 valori mancanti, ma adesso ne ha zero. Questo potrebbe essere dovuto al fatto che gli unici due valori mancanti appartenevano alle auto elettriche, eliminate dal dataset in precedenza. Questo perché l'informazione di Mileage, ovvero *km/quantità_carburante*, per un auto elettrica non ha senso.

*Correzzione di Inconsistenze e Formattazione*

Adesso verifichiamo se tutte le osservazioni sono conformi con la formattazione e condividono la stessa unità di misura:

In [223]:
df_mileage = df['Mileage'].dropna()

# Controllo sulla formattazione
pattern = r'^\S+\s\S+$'

risultato_controllo = df_mileage.astype(str).str.match(pattern)

total_obs = len(df_mileage)
conformi = risultato_controllo.sum()
non_conformi = total_obs - conformi

print(f"Conformi: {conformi}")
print(f"Non conformi: {non_conformi}")

# Controllo sulle unità
mileage_units = df_mileage.str.split().str[-1]

print("\n---- Unità di misura presenti ----")
mileage_units.unique()

Conformi: 6017
Non conformi: 0

---- Unità di misura presenti ----


array(['km/kg', 'kmpl'], dtype=object)

Per nostra fortuna tutte le righe **seguono la formattazione prevista**, tuttavia vengono usate **due differenti unità di misura**: 'km/kg' e 'kmpl' (chilometri su chulogrammo e chulometri al litro).

Queso potrebbe essere perché differenti tipi di carburante (`Fuel_Type`) vengono misuranti in modo differente e quindi pure la misura di `Mileage` deve adattarsi.

Indaghiamo meglio:

In [224]:
print(mileage_units.value_counts())
print("")

print(df['Fuel_Type'].value_counts())
print("")

cross_tab = pd.crosstab(mileage_units, df['Fuel_Type'])
print(cross_tab)

Mileage
kmpl     5951
km/kg      66
Name: count, dtype: int64

Fuel_Type
Diesel      3205
Petrol      2746
CNG           56
LPG           10
Electric       2
Name: count, dtype: int64

Fuel_Type  CNG  Diesel  LPG  Petrol
Mileage                            
km/kg       56       0   10       0
kmpl         0    3205    0    2746


Notiamo varie cose:
- le osservazioni che utilizzano 'km/kg' come unità per `Mileage` sono molto poche
- per i carburanti l'uso delle unità di misura per `Mileage` sono consistenti: nessun tipo di carburante (`Fuel_Type`) utilizza entrambe le unità per le sue osservazioni
- le osservazioni che utilizzano 'km/kg' per `Mileage` (66) coincidono con quelle che utilizzano i due tipi di carburante (`Fuel_Type`) meno presenti (CNG 56, LPG 10)

Potremmo rimuovere osservazioni che utilizzano differenti unità per `Mileage`, che essendo molto poche, risultano irrilevanti. Oppure possiamo uniformare le unità di misura.

Per il momento **manteniamo queste osservazioni** e le uniformeremo secondo l'**unità predominante**, ovvero 'kmpl'. I due carburanti che utilizzno 'km/kg' in Mileage sono LPG e CNG, che sono gas naturali, tipicamente 1kg di gas è pari a circa 1.4 litri di carburante liquido, il fattore di conversione si basa sulla **densità energetica**.

Passiamo al processo di pulizia di `Mileage`:

In [225]:
factor = 1 / 1.4

df_clean['Mileage_Unit'] = df_clean['Mileage'].str.split().str[-1]
df_clean['Mileage'] = df_clean['Mileage'].str.split(' ').str[0]
df_clean['Mileage'] = pd.to_numeric(df_clean['Mileage'], errors='coerce')

df_clean.loc[df_clean['Mileage_Unit'] == 'km/kg', 'Mileage'] *= factor

df_clean.drop('Mileage_Unit', axis=1, inplace=True)

print(df_clean.isna().sum())

Brand                   0
Model                   0
Location                0
Age                     0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 0
Engine                 36
Power                  36
Seats                  42
New_Price            5194
Price                   0
dtype: int64


**`Engine`**

> Contene un numero continuo.

Verifichiamo il numero di valori mancanti:

In [226]:
print("Valori mancanti:", df_clean['Engine'].isna().sum())


Valori mancanti: 36
